<a href="https://colab.research.google.com/github/stemgene/Predict-Medical-Appointment-No-Shows/blob/master/Machine_learning_based_on_the_no_show_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction:

In this kernel, we tried to make the classification of label of "no show" as the processes below:

* Data preprocessing
* Split data, and address over-sampling using SMOTE
* Use Logistic Regresson, Random Forest, and SVM to train the model
* Use neural network to train the model

### Result:
Model performance was evaluated by precision, recall, and accuracy. The table below summarizes the model performance on the validation and testing set. As can be seen from this table, logistic regression and SVM did not perform well on our sample, even after hyperparameter tuning. Random forest model produced the best results, with a precision of 0.83, a recall of 0.73, and an accuracy of 0.91 on the testing set. Although the neural network resulted in a good performance on the validation set after fine-tuning, the precision and recall decreased to 0.47 at testing time. Results of the sensitivity analysis were included in the Appendix, where we did not observe significant differences from our main analysis.

<table>
  <tr>
    <th>  </th>
    <th colspan='2'><center>Linear Regression</center></th>
    <th colspan='2'><center>SVM</center></th>
    <th colspan='2'><center>Random Forest</center></th>
    <th colspan='2'><center>Neural Network</center></th>
  </tr>
  <tr>
    <td> Dataset </td>
    <td>Validation Set</td>
    <td>Testing Set</td>
    <td>Validation Set</td>
    <td>Testing Set</td>
    <td>Validation Set</td>
    <td>Testing Set</td>
    <td>Validation Set</td>
    <td>Testing Set</td>
  </tr>
  <tr>
    <td>Precision</td>
    <td><center>0.32</center></td>
    <td><center>0.31</center></td>
    <td><center>0.29</center></td>
    <td><center>0.3</center></td>
    <td><center>0.82</center></td>
    <td><center>0.83</center></td>
    <td><center>0.84</center></td>
    <td><center>0.47</center></td>
  </tr>
  <tr>
    <td>Recall</td>
    <td><center>0.59</center></td>
    <td><center>0.59</center></td>
    <td><center>0.88</center></td>
    <td><center>0.89</center></td>
    <td><center>0.72</center></td>
    <td><center>0.73</center></td>
    <td><center>0.78</center></td>
    <td><center>0.47</center></td>
  </tr>
  <tr>
    <td>Accuracy</td>
    <td><center>0.66</center></td>
    <td><center>0.66</center></td>
    <td><center>0.54</center></td>
    <td><center>0.54</center></td>
    <td><center>0.91</center></td>
    <td><center>0.91</center></td>
    <td><center>0.82</center></td>
    <td><center>0.78</center></td>
  </tr>
</table>

We performed variable importance analysis with logistic regression and random forest. The results are shown in the table below. These two models produced consistent variable importance, indicating that lead time, age, previous no-show rate, receiving reminder messages, and male gender are strong predictors of no-show outcome. Specifically, from Table 3 we can see that patients with long lead time, with diabetes and alcoholism, and having a high previous no-show rate were more likely to be no-show patients. Male patients and older patients were more likely to attend their appointments. Receiving reminder messages was positively associated with the outcome, one interpretation is that the reminder messages may only have been sent to patients who had nonattendance history.

<table>
  <tr>
    <th><center>Variables</center></th>
    <th><center>Coef.</center></th>
    <th><center>p_value</center></th>
  </tr>
  <tr>
    <td>Lead_Time</td>
    <td>0.0271</td>
    <td>0</td>
  </tr>
  <tr>
    <td>Diabetes</td>
    <td>0.1022</td>
    <td>0.0004</td>
  </tr>
  <tr>
    <td>Alcoholism</td>
    <td>0.2656</td>
    <td>0</td>
  </tr>
  <tr>
    <td>SMS_Received</td>
    <td>0.3538</td>
    <td>0</td>
  </tr>
  <tr>
    <td>Male</td>
    <td>-0.1676</td>
    <td>0</td>
  </tr>
  <tr>
    <td>Age_scaled</td>
    <td>-1.3964</td>
    <td>0</td>
  </tr>
  <tr>
    <td>Noshow_Rate</td>
    <td>-0.6419</td>
    <td>0</td>
  </tr>
</table>

In [0]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

df = pd.read_csv("../input/noshowappointments/KaggleV2-May-2016.csv", )

# Data Preprocessing

We summarized all features into six categories. The features of “patient ID” and “appointment ID” identify a patient and a record of the appointment. The patient’s demographic category shows gender, neighbourhood, and age. There are two timestamp variables, “scheduled day” and “appointment day”. We also have some binary data, which indicate patients’ medical conditions, insurance, and whether the patient receives a reminder message from the hospital. As a dataset of classification, the label feature is “No Show”, which is a binary variable, it shows whether the patient is no-show or not.
* Index: PatientId, AppointmentID
* Categorical: Gender, Neighborhood
* Time stamp: ScheduledDay, AppointmentDay
* Numerical: Age
* Binary: Alcoholism, Hipertension, Scholarship, Diabetes, Handcap, SMS_received
* Label: No-show

### 1. ScheduledDay and AppointmentDay

previous literature indicates that lead time (time interval between schedule day and the actual appointment day) is one of the strong predictors of no-show, we thus constructed this variable using the variables ScheduleDay and AppointmentDay in our dataset. Another important predictor is the day of week, which may affect patient’s behavior. Therefore, we created a new feature “day of week” using the variable AppointmentDay.    

In [0]:
# Insert the 5th column that indicate the weekday of AppointmentDay
# dt.weekday: Monday-0, Friday-4, Sunday-6
df.insert(5, "Appoint Weekday", df.loc[:, 'AppointmentDay'].apply(np.datetime64).dt.weekday)

# Transform str to pd.datestamp and date form
df.loc[:, 'ScheduledDay'] = df.loc[:, 'ScheduledDay'].apply(np.datetime64).dt.date
df.loc[:, 'AppointmentDay'] = df.loc[:, 'AppointmentDay'].apply(np.datetime64).dt.date

# Insert the 6th column that indicate the difference between scheduled date and appointment date
date_difference = []
for i in range(len(df.loc[:, 'ScheduledDay'])):
  diff = (df.loc[:, 'AppointmentDay'][i]-df.loc[:, 'ScheduledDay'][i]).days
  date_difference.append(diff)
df.insert(6, "Date Difference", date_difference)

### 2. Location Information: "Neighborhood"

Patients’ geographical information was specified to the town level in our dataset. There are 81 towns in Victoria, a city in Brazil. 

We got every town's GPS address from Google Map API, however, if Google cannot search information by the input, it will return the address of upper level, which is the state of ES, Brazil. We had to pick these wrong results and check them by hand.

In this notebook, we omit this session, instead, we upload the file containing the GPS information and frequency of appointment of every town.

In [0]:
import pandas as pd
towns_df = pd.read_csv("../input/no-show/towns.csv")
towns = {row[0]:[row[1],float(row[2]),float(row[3])] for index, row in towns_df.iterrows()} # load file is string type, need to transform float
len(towns)  #79, already drop 2 observations

In [0]:
cluster_df = df[["PatientId", 'Neighbourhood']]
# delete extreme values
cluster_df.drop(cluster_df.index[cluster_df['Neighbourhood'] == 'ILHAS OCEÂNICAS DE TRINDADE'], inplace = True)
cluster_df.drop(cluster_df.index[cluster_df['Neighbourhood'] == 'PARQUE INDUSTRIAL'], inplace = True)
whole_towns_gps = [(towns[key][1], towns[key][2]) for key in cluster_df['Neighbourhood'].values]
cluster_df.insert(2,"town gps", whole_towns_gps)
cluster_df.tail(2)

#### visualize the patient distribution

If we zoom in the figure shown below, we can find many towns contain less than 500 patients. In total of 81 towns, we want to merge small towns into adjacent towns and combine them into larger groups.

In [0]:
!pip install pyepsg

In [0]:
import geopandas as gpd
import pyepsg
import folium
from folium.plugins import HeatMap
from folium.plugins import MarkerCluster
locationlist = [(v[0],v[1]) for v in cluster_df['town gps'].values]
map2 = folium.Map(location=[-20.268857, -40.302106], tiles='cartodbdark_matter', zoom_start=13)
marker_cluster = MarkerCluster().add_to(map2)

for point in range(0, 5000):  # The maximum number is 5000, if the number is greater, this map won't show up. Anyone can tell me what's wrong with it?
    folium.Marker(locationlist[point]).add_to(marker_cluster)
map2

#### K-means


We use K-means to merge nearby towns automatically.

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()  # for plot styling
import numpy as np
from matplotlib.pyplot import figure
from sklearn.cluster import KMeans

X = np.array([[v[1], v[2]] for v in towns.values()])
kmeans = KMeans(n_clusters=30)
kmeans.fit(X)
y_kmeans = kmeans.predict(X)

figure(num=None, figsize=(15, 12), dpi=80, facecolor='w', edgecolor='k')
plt.scatter(X[:, 1], X[:, 0], c=y_kmeans, s=30, cmap='viridis')

We certainly can introduce K-means like this. However, the result is not good enough, since this method just merge points depend on their locations, but ignore the importance or count number of patients of a town. If we compare it with the first figure above, some small towns such as "ILHA DO FRADE", which is an island, are clusters alone. 

We want to merge these towns not only by distance, but also by the weight or count number of patients.

Therefore we synthesize virtual points for each town according the count number of patients, and apply K-means again.

In [0]:
# synthesize virtual points for each town according the count number of patients
# distance of 0.001(gps) = 110m
np.random.seed(0)
cov = ([-3e-8, 0],[0, 3e-8]) # range of random
patient_gps = [np.random.multivariate_normal(v,  cov , 1) for v in cluster_df['town gps'].values]
cluster_df.insert(3,"patient gps", patient_gps)
cluster_df.tail(2)

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()  # for plot styling
import numpy as np
from matplotlib.pyplot import figure
from sklearn.cluster import KMeans

X = np.array([(v[0][0],v[0][1]) for v in cluster_df['patient gps'].values])
kmeans = KMeans(n_clusters=15)
kmeans.fit(X)
y_kmeans = kmeans.predict(X)

figure(num=None, figsize=(15, 12), dpi=80, facecolor='w', edgecolor='k')
plt.scatter(X[:, 1], X[:, 0], c=y_kmeans, s=30, cmap='viridis', edgecolor='None')

We can see small towns are merged into other groups. 

Finally we got 15 clusters, and insert this column to dataframe.

In [0]:
drop=['ILHAS OCEÂNICAS DE TRINDADE', 'PARQUE INDUSTRIAL'] #delete two extreme values, which cannot be found on Google map
df = df[~df['Neighbourhood'].isin(drop)]
df.insert(9, "K-means cluster", y_kmeans)
#delete some wrong data from last session
df = df[df["Date Difference"] >= 0] 
df.tail()

### 3. One-hot coding

In [0]:
df['Male'] = np.where(df['Gender']== "M", 1, 0)
df['Label'] = np.where(df['No-show']=="Yes", 1, 0)
cat_columns = ['Handcap','Appoint Weekday', 'K-means cluster']
df = pd.get_dummies(df, prefix_sep="__", columns=cat_columns)

### 4.Age

we removed patients under 0-year-old and normalized the variable.

In [0]:
# 28866(26%) patients are under 18, probably not appropriate to drop them
df['under18'] =  np.where(df['Age'].astype(int)>18, "no", "yes" )
print(df['under18'].value_counts())

# delete rows with age<0
df.drop(df[df['Age'].astype(int) < 0].index, inplace = True) 
hist = df['Age'].astype(int).hist(bins=100)
hist

# normalize (instead of grouping, i remember back in class jack said grouping age will cause information loss)
from sklearn import preprocessing
age = df[['Age']].values.astype(int)
min_max_scaler = preprocessing.MinMaxScaler()
df['age_scaled'] =  min_max_scaler.fit_transform(age)
 
boolean = any(df.duplicated())
print("any duplicates: " , boolean)

### 5. Noshow History

From some literature, a patient’s no-show history would be a very important feature. Hence we calculated the no-show rate before current patient visit based on “Patient ID” and “No-show” label together. Noteworthily, we created a new feature to distinguish new patients with other patients who truly had zero no-show rate for previous visits (clean sheet).

In [0]:
## check
df['Label'].value_counts()
## distinct PatientId count
len(df['PatientId'].unique())  # 62296

df_visit=df.sort_values(by=['PatientId','ScheduledDay'])

##  visit
df_visit['Visit_Nth']=df_visit.groupby('PatientId').cumcount()+1
df_visit['New_Visit']=df_visit['Visit_Nth'].apply(lambda x: 1 if x==1 else 0)
df_visit['Noshow_cumsum']=df_visit.groupby('PatientId')['Label'].cumsum()

### noshow rate 
def noshow_rate(df):
    if df['Visit_Nth']==1:
        rate = 0    
    elif df['Visit_Nth']>1:
        if df['Label']==1:
            rate = (df['Noshow_cumsum']-1 )/ (df['Visit_Nth'] -1 )
        else:
            rate = (df['Noshow_cumsum']  )/ (df['Visit_Nth'] -1 )       
    return rate

df_visit['Noshow_rate']= df_visit.apply(noshow_rate, axis=1)

## max
df_max= pd.DataFrame(df_visit.groupby('PatientId')['Visit_Nth'].max()).reset_index()
df_max = df_max.rename(columns={'Visit_Nth':'Visit_Nth_max'} )
df_max.head()

## check unique patients
print(df_max.shape,"\n",df_max['Visit_Nth_max'].describe())
df_max['Visit_Nth_max'].value_counts()

## hist
df_max.hist(bins=20,label='Visit_Nth_max');
plt.xlabel("Visit_Nth_max")
plt.ylabel("Patients")
plt.title("Distribution of Visit_Nth_max")
plt.show()

## outlier defined
cut_list = [0, 0.25, 0.5, 0.75, 0.85, 0.9, 0.95, 0.99, 1]
cut_Visit_Nth_max = df_max['Visit_Nth_max'].quantile(q=cut_list)
print(cut_Visit_Nth_max)
## decide to drop top 1%: keep Visit_Nth_max <=8

## merge
df_fin= df_visit.merge(df_max, on='PatientId', how='left')
df_fin.shape

## check
df_fin.tail(10)

## check
(df_fin[df_fin.Noshow_rate>0 & (df_fin.Noshow_rate<1)]).tail(10)

## check
df_fin[df_fin.PatientId=='99966898398164']

df= df_fin.copy()

## drop outlier
df= df[df['Visit_Nth_max']<=8]

## only keep Noshow_rate and New_Visit
df = df.drop(['Visit_Nth','Noshow_cumsum','Visit_Nth_max'], axis=1)

boolean = any(df.duplicated())
print("any duplicates: " , boolean)

# Splitting data by PatientId, Over-sampling using SMOTE

There were 22,319 (20.19%) missed appointments in our dataset, indicating the issue of class imbalance. We doubled the observations in the minority class and randomly sampled an equal number of observations in the majority class to address this issue.

In [0]:
from sklearn.model_selection import GroupShuffleSplit
from imblearn.over_sampling import SMOTE
import seaborn as sns

# splitting by patientid
_, id_indices = np.unique(np.array(df['PatientId']), return_inverse=True)
train_ids, val_test_ids = next(GroupShuffleSplit(test_size=.30, n_splits=2, random_state = 8).split(df, groups=id_indices))

val_test_set = df.iloc[val_test_ids]
_, id_indices = np.unique(np.array(val_test_set['PatientId']), return_inverse=True)
val_ids, test_ids = next(GroupShuffleSplit(test_size=.50, n_splits=2, random_state = 8).split(val_test_set , groups=id_indices))

train_set = df.iloc[train_ids]
val_set =  df.iloc[val_ids]
test_set =  df.iloc[test_ids]

boolean = any(train_set.duplicated())
print("train_set any duplicates: " , boolean)

boolean = any(val_set.duplicated())
print("val_set any duplicates: " , boolean)

boolean = any(test_set.duplicated())
print("test_set any duplicates: " , boolean)
      
# drops = ['No-show',"Gender",'AppointmentID', 'PatientId', 'ScheduledDay','AppointmentDay',  'Age','under18', 'Neighbourhood']
# train_set = train_set.drop(drops, axis=1)
# val_set = val_set.drop(drops, axis=1)
# test_set = test_set.drop(drops, axis=1)

# Check whether there are any duplicate values in different data sets.
check =  any(item in train_set['PatientId'] for item in (val_set['PatientId']))
if check:
  print("train_set['PatientId'] contains some elements of val_set['PatientId']")      
else:
  print("train_set['PatientId'] does not have any element of val_set['PatientId'].")

check =  any(item in train_set['PatientId'] for item in (test_set['PatientId']))
if check:
  print("train_set['PatientId'] contains some elements of test_set['PatientId']")      
else:
  print("train_set['PatientId'] does not have any element of test_set['PatientId'].")

check =  any(item in val_set['PatientId'] for item in (test_set['PatientId']))
if check:
  print("val_set['PatientId'] contains some elements of test_set['PatientId']")      
else:
  print("val_set['PatientId'] does not have any element of test_set['PatientId'].")

boolean = any(train_set.duplicated())
print("train_set any duplicates: " , boolean)

boolean = any(val_set.duplicated())
print("val_set any duplicates: " , boolean)

boolean = any(test_set.duplicated())
print("test_set any duplicates: " , boolean)

drops = ['Label','No-show',"Gender",'AppointmentID', 'PatientId', 'ScheduledDay','AppointmentDay', 'Age','under18', 'Neighbourhood']
X_train = train_set.drop(drops, axis=1, index=None)
y_train = train_set['Label']

X_val = val_set.drop(drops, axis=1)
y_val = val_set['Label']

X_test = test_set.drop(drops, axis=1)
y_test = test_set['Label']

#### Resample the minority class

In [0]:
sm = SMOTE(sampling_strategy='minority',random_state=8)


X_train_os, y_train_os = sm.fit_sample(X_train, y_train)  
X_train_os = pd.DataFrame(X_train_os)
X_train_os.columns = X_train.columns
y_train_os = pd.DataFrame(y_train_os)
y_train_os.columns = ['Label']
boolean = any(X_train_os.duplicated())
print("\nX_train_os any duplicates: " , boolean)



X_val_os, y_val_os = sm.fit_sample(X_val, y_val)
X_val_os = pd.DataFrame(X_val_os)
X_val_os.columns = X_val.columns
y_val_os = pd.DataFrame(y_val_os)
y_val_os.columns = ['Label']


X_test_os, y_test_os = sm.fit_sample(X_test, y_test)
X_test_os = pd.DataFrame(X_test_os)
X_test_os.columns = X_test.columns
y_test_os = pd.DataFrame(y_test_os)
y_test_os.columns = ['Label']


train_set_os = pd.concat([X_train_os, y_train_os], axis=1)
val_set_os = pd.concat([X_val_os, y_val_os], axis=1)
test_set_os = pd.concat([X_test_os, y_test_os], axis=1)

print('\ntrain_set: ',train_set.shape, "val_set: ", val_set.shape, "test_set:", test_set.shape)
print('\ntrain_set_os: ',train_set_os.shape, "val_set_os: ", val_set_os.shape, "test_set_os:", test_set_os.shape)

boolean = any(train_set_os.duplicated())
print("train_set_os any duplicates: " , boolean)

boolean = any(val_set_os.duplicated())
print("val_set_os any duplicates: " , boolean)

boolean = any(test_set.duplicated())
print("test_set_os any duplicates: " , boolean)

# Use Logistic Regresson, Random Forest, and SVM to train the model

In [0]:
features  = ['Date Difference', 'Scholarship', 'Hipertension', 'Diabetes',
       'Alcoholism', 'SMS_received', 'Male', 'Handcap__0',
       'Handcap__1', 'Handcap__2', 'Handcap__3', 'Handcap__4',
       'Appoint Weekday__0', 'Appoint Weekday__1', 'Appoint Weekday__2',
       'Appoint Weekday__3', 'Appoint Weekday__4', 'Appoint Weekday__5',
       'K-means cluster__0', 'K-means cluster__1', 'K-means cluster__2', 'K-means cluster__3', 'K-means cluster__4',
       'K-means cluster__5', 'K-means cluster__6', 'K-means cluster__7', 'K-means cluster__8', 'K-means cluster__9',
       'K-means cluster__10', 'K-means cluster__11', 'K-means cluster__12', 'K-means cluster__13',
       'K-means cluster__14', 'age_scaled', 'New_Visit', 'Noshow_rate']


X_train_os = train_set_os[features]
y_train_os = train_set_os['Label']

X_val_os = val_set_os[features]
y_val_os = val_set_os['Label']


X_train = train_set[features]
y_train = train_set['Label']

X_val = val_set[features]
y_val = val_set['Label']

X_test = test_set[features]
y_test = test_set['Label']

#### Logistic Regression

In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix
from datetime import datetime


def lr(X, y):
  time_start = datetime.now()
  name =[x for x in globals() if globals()[x] is X][0]

  logreg = LogisticRegression()
  logreg.fit(X, y)
  y_pred = logreg.predict(X_test)
  cm = confusion_matrix(y_test, y_pred)
  print(f'\n\n Accuracy of logistic regression classifier on test set: {round(logreg.score(X_test, y_test),2)} -- { name}')
  print(cm)
  print(classification_report(y_test, y_pred))

  time_end = datetime.now()
  time_total = (time_end - time_start).total_seconds()
  print("\n run time: ", time_total, "\n")


lr(X_train, y_train)
lr(X_train_os, y_train_os)

#### Random Forest

In [0]:
from sklearn.ensemble import RandomForestClassifier


def rf(X, y):
  time_start = datetime.now()
  name =[x for x in globals() if globals()[x] is X][0]

  rf = RandomForestClassifier()
  rf.fit(X, y)
  y_pred = rf.predict(X_test)
  print(f"\n\n random forest -- {name}")
  print(confusion_matrix(y_test, y_pred))
  print(classification_report(y_test, y_pred))
  fi = pd.DataFrame({'feature': list(X.columns),
                   'importance': rf.feature_importances_}).\
                    sort_values('importance', ascending = False)
  print("feature importance: ", fi.head(10))
  
  time_end = datetime.now()
  time_total = (time_end - time_start).total_seconds()
  print("\n run time: ", time_total, "\n")



rf(X_train, y_train)
rf(X_train_os, y_train_os)

#### SVM

Be careful, the algorithm of SVM will take so long time to run with a large dataset

In [0]:
from sklearn.svm import SVC

def svm(X, y, kernel):
  time_start = datetime.now()
  name =[x for x in globals() if globals()[x] is X][0]
  
  df = pd.concat([X, y], axis=1).sample(frac = .5, random_state = 8)
  X = df[features]
  y = df['Label']  

  svclassifier = SVC(kernel=kernel)
  svclassifier.fit(X, y)
  y_pred=svclassifier.predict(X_test)
  print(f'\n\n  SVM: {kernel} kernel -- {name} \n {confusion_matrix(y_test, y_pred)}')
  print(classification_report(y_test, y_pred))

  time_end = datetime.now()
  time_total = (time_end - time_start).total_seconds()
  print("\n run time: ", time_total, "\n")



# svm(X_train, y_train, "linear")
# svm(X_train_os, y_train_os, "linear")

svm(X_train, y_train, "poly")
svm(X_train_os, y_train_os, "poly")


svm(X_train, y_train, "rbf")
svm(X_train_os, y_train_os, "rbf")

svm(X_train, y_train, "sigmoid")
svm(X_train_os, y_train_os, "sigmoid")

# Use Neural Network to train the model

Training session. Tune the hyperparameters with validation set

In [0]:
import pandas as pd
import numpy as np
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))  

import torch
from torch import nn, optim
import torch.nn.functional as F
import torch.utils.data as data
import torchvision
import os
import numpy as np
import pandas as pd
from datetime import datetime
import sys

"""Setup the network"""
class mydata(data.Dataset):
   
    def __init__(self, df, label_name):
        #df = pd.read_csv(data_file_path)
        labels = df[label_name].values
        data = df.drop(label_name, axis=1)
        self.datalist = data
        self.labels = labels

    def __getitem__(self, index):
        return torch.Tensor(np.asarray(self.datalist.iloc[[index]].astype(float))), self.labels[index]

    def __len__(self):
        return self.datalist.shape[0]


class Network(nn.Module):
    def __init__(self, dropout_rate = 0.02):
        super().__init__()      
        self.hidden1 = nn.Linear(36, 25)     
        self.hidden2 = nn.Linear(25, 10)      
        self.output = nn.Linear(10, 2)

        self.sigmoid1 = nn.Sigmoid()
        self.sigmoid2 = nn.Sigmoid()
        # self.sigmoid =nn.Sigmoid()
        self.softmax = nn.LogSoftmax(dim=1)

        self.dropout1 = nn.Dropout(p=dropout_rate)
        self.dropout2 = nn.Dropout(p=dropout_rate)
      
        self.batchnorm1 = nn.BatchNorm1d(25)
        self.batchnorm2 = nn.BatchNorm1d(10)
   
        
    def forward(self, x):
        x = x.view(x.shape[0], -1)
        x = self.hidden1(x)
        x = self.sigmoid1(x)
        x = self.dropout1(x)
        x = self.batchnorm1(x)

        x = self.hidden2(x)
        x = self.sigmoid2(x)
        x = self.dropout2(x)
        x = self.batchnorm2(x)

        x = self.output(x)
        x = self.softmax(x)
        # x = self.sigmoid(x)    
        return x
    
"""hyperparameters"""
loss_func = nn.NLLLoss()
loss_func_name = 'neg log like'

# loss_func = nn.CrossEntropyLoss()
# loss_func_name = 'cross entropy'
learn_rate = .001
num_epochs = 1000
batch_size = 1024
confidence_threshold = 0.5
loss_adj_conf_thresh = np.log(confidence_threshold)
optimizer_name = 'Adam'
start_time = datetime.now()

"""export training log"""
run_id = f"hp run_{datetime.now().hour}_{datetime.now().minute}"
try:
    os.mkdir(run_id)
except FileExistsError:
    pass


with open(run_id + '/hyperparams.csv', 'w') as wfil:
    wfil.write("loss function," + loss_func_name + '\n')
    wfil.write("learning rate," + str(learn_rate) + '\n')
    wfil.write("number epochs," + str(num_epochs) + '\n')
    wfil.write("batch size," + str(batch_size) + '\n')
    wfil.write("optimizer," + str(learn_rate) + '\n')
    wfil.write("start time," + str(start_time) + '\n')

"""load data"""
train_set = mydata(train_set_os, 'Label')
trainloader = torch.utils.data.DataLoader(
    dataset=train_set, batch_size=batch_size, shuffle=True)
valid_set = mydata(val_set_os, 'Label')
validloader = torch.utils.data.DataLoader(
    dataset=valid_set, batch_size=batch_size, shuffle=True)

"""detect device, this code can be used on GPU """
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Network()

if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    model = nn.DataParallel(model)
model = model.to(device)    

print('device is :', device)

optimizer = optim.Adam(model.parameters(), lr=learn_rate)  # The optimizer is Adam 

best_valid_loss = "unset"

"""Training"""
with open(run_id + '/log_file.csv', 'w') as log_fil: 
    
    log_fil.write(
        "epoch,epoch duration,train loss,valid loss,test loss,train accuracy, valid accuracy, test accuracy,  train precision, valid precision, test precision, train recall, valid recall, test recall\n")

    for epoch in range(0, num_epochs):
        epoch_start = datetime.now()
   
        epoch_train_loss = 0.0
        epoch_valid_loss = 0.0
    
        epoch_train_accuracy = 0.0
        epoch_valid_accuracy = 0.0
        epoch_train_counter = 0.0
        epoch_valid_counter = 0.0
           
        epoch_train_tp = 0.0
        epoch_valid_tp = 0.0
      
        epoch_train_tn = 0.0
        epoch_valid_tn = 0.0      

        epoch_train_fp = 0.0
        epoch_valid_fp = 0.0

        epoch_train_fn = 0.0
        epoch_valid_fn = 0.0
      
        epoch_train_precision = 0.0
        epoch_valid_precision = 0.0      

        epoch_train_recall = 0.0
        epoch_valid_recall = 0.0
       

        for i, (images, labels) in enumerate(trainloader):
            
            images = images.to(device)
            labels = labels.to(device)           
            optimizer.zero_grad()
            outputs = model(images)
#             print("Outside: input size", labels.size(), "output_size", outputs.size())         
            loss = loss_func(outputs, labels)     
            loss.backward()          
            optimizer.step()
 
            epoch_train_loss += loss.item()

            
            for i in range(0, len(labels)):
                temp_label = labels[i]
                temp_pred = outputs[i, 1]
                
                if temp_pred > loss_adj_conf_thresh:
                    temp_pred = 1.0
                else:
                    temp_pred = 0.0
                
                if float(temp_pred) - float(temp_label) == 0:
                    epoch_train_accuracy += 1.0

                epoch_train_counter += 1.0

                if float(temp_label) == 1.0 and float(temp_pred) == 1.0:
                    epoch_train_tp += 1.0

                if float(temp_label) == 0.0 and float(temp_pred) == 0.0:
                    epoch_train_tn += 1.0

                if float(temp_label) == 0.0 and float(temp_pred) == 1.0:
                    epoch_train_fp += 1.0

                if float(temp_label) == 1.0 and float(temp_pred) == 0.0:
                    epoch_train_fn += 1.0

        epoch_train_accuracy = epoch_train_accuracy / epoch_train_counter
        try:
            epoch_train_precision = epoch_train_tp / \
                (epoch_train_tp + epoch_train_fp)
        except ZeroDivisionError:
            epoch_train_precision = None

        try:
            epoch_train_recall = epoch_train_tp / \
                (epoch_train_tp + epoch_train_fn)
        except ZeroDivisionError:
            epoch_train_recall = None
            
 
        
        with torch.no_grad():
            for i, (images, labels) in enumerate(validloader):
                images = images.to(device)
                labels = labels.to(device)
                outputs = model(images)
                loss = loss_func(outputs, labels)
                epoch_valid_loss += loss.item()

                
                for i in range(0, len(labels)):
                    temp_label = labels[i]
                    temp_pred = outputs[i, 1]
                    
                    if temp_pred > loss_adj_conf_thresh:
                        temp_pred = 1.0
                    else:
                        temp_pred = 0.0

                    if float(temp_pred) - float(temp_label) == 0:
                        epoch_valid_accuracy += 1.0
                    epoch_valid_counter += 1.0

                    if float(temp_label) == 1.0 and float(temp_pred) == 1.0:
                        epoch_valid_tp += 1.0

                    if float(temp_label) == 0.0 and float(temp_pred) == 0.0:
                        epoch_valid_tn += 1.0

                    if float(temp_label) == 0.0 and float(temp_pred) == 1.0:
                        epoch_valid_fp += 1.0

                    if float(temp_label) == 1.0 and float(temp_pred) == 0.0:
                        epoch_valid_fn += 1.0

        epoch_valid_accuracy = epoch_valid_accuracy / epoch_valid_counter
        try:
            epoch_valid_precision = epoch_valid_tp / \
                (epoch_valid_tp + epoch_valid_fp)
        except ZeroDivisionError:
            epoch_valid_precision = None
        try:
            epoch_valid_recall = epoch_valid_tp / \
                (epoch_valid_tp + epoch_valid_fn)
        except ZeroDivisionError:
            epoch_valid_recall = None
                    
                     
        epoch_end = datetime.now()
        epoch_time = (epoch_end - epoch_start).total_seconds()

        
        if best_valid_loss == "unset" or epoch_valid_loss < best_valid_loss:
            best_valid_loss = epoch_valid_loss
            torch.save(model, run_id + "/best_weights.pth")

        
        torch.save(model, run_id + "/last_weights.pth")

        
        log_fil.write(str(epoch) + ',' + str(epoch_time) + ','
                      + str(epoch_train_loss) + ','
                      + str(epoch_valid_loss) + ','
                      
                      + str(epoch_train_accuracy) + ','
                      + str(epoch_valid_accuracy) + ','
                      
                      + str(epoch_train_precision) + ','
                      + str(epoch_valid_precision) + ','
                      
                      + str(epoch_train_recall) + ','
                      + str(epoch_valid_recall) 
                     
                      + '\n')

       
        print("epoch: " + str(epoch) + " - (" + str(epoch_time) + " seconds)" + "\n\ttrain loss: " + str(epoch_train_loss)[:5] 
              + " - train accuracy: " + str(epoch_train_accuracy)[:5]  
              + " - train precision " + str(epoch_train_precision)[:5]  
              + " - train recall: " + str(epoch_train_recall)[:5]  
              

              + "\n\tvalid loss: " + str(epoch_valid_loss)[:5]  
              + " - valid accuracy: " + str(epoch_valid_accuracy)[:5] 
              + " - valid precision " + str(epoch_valid_precision)[:5]  
              + " - valid recall: " + str(epoch_valid_recall)[:5]  
         
             )                                                    

end_time = datetime.now()
with open(run_id + '/hyperparams.csv', 'a') as wfil:
    wfil.write("end time," + str(end_time) + '\n')

Get final results by test set

In [0]:
import torch
from torch import nn, optim
import torch.nn.functional as F
import torch.utils.data as data
import torchvision
import os
import numpy as np
import pandas as pd
from datetime import datetime
import sys

class mydata(data.Dataset):
   
    def __init__(self, df, label_name):
        labels = df[label_name].values
        data = df.drop(label_name, axis=1)
        self.datalist = data
        self.labels = labels

    def __getitem__(self, index):
        return torch.Tensor(np.asarray(self.datalist.iloc[[index]].astype(float))), self.labels[index]

    def __len__(self):
        return self.datalist.shape[0]


class Network(nn.Module):
    def __init__(self, dropout_rate = 0.02):
        super().__init__()      
        self.hidden1 = nn.Linear(36, 25)     
        self.hidden2 = nn.Linear(25, 10)      
        self.output = nn.Linear(10, 2)

        self.sigmoid1 = nn.Sigmoid()
        self.sigmoid2 = nn.Sigmoid()
        # self.sigmoid =nn.Sigmoid()
        self.softmax = nn.LogSoftmax(dim=1)

        self.dropout1 = nn.Dropout(p=dropout_rate)
        self.dropout2 = nn.Dropout(p=dropout_rate)
      
        self.batchnorm1 = nn.BatchNorm1d(25)
        self.batchnorm2 = nn.BatchNorm1d(10)
   
        
    def forward(self, x):
        x = x.view(x.shape[0], -1)
        x = self.hidden1(x)
        x = self.sigmoid1(x)
        x = self.dropout1(x)
        x = self.batchnorm1(x)

        x = self.hidden2(x)
        x = self.sigmoid2(x)
        x = self.dropout2(x)
        x = self.batchnorm2(x)

        x = self.output(x)
        x = self.softmax(x)
        # x = self.sigmoid(x)    
        return x
    
# identify where the weights you want to load are 
weight_fil = "../input/no-show/best_weights.pth"

# set necessary hyperparameters
batch_size = 1024
loss_func = nn.NLLLoss()
confidence_threshold = 0.5
loss_adj_conf_thresh = np.log(confidence_threshold)


# # initialize model
# model = Network()

# # load weights
# model.load_state_dict(torch.load(weight_fil))

model = torch.load(weight_fil)

# put model in evaluation mode (sets dropout and batch normalization layers to evaluation mode before running inference. Failing to do this will yield inconsistent inference results)
model.eval()

# create loaders to feed in data to the network in batches
drops = ['No-show',"Gender",'AppointmentID', 'PatientId', 'ScheduledDay','AppointmentDay', 'Age','under18', 'Neighbourhood']
test_set = test_set.drop(drops, axis=1).copy()
eval_set = mydata(test_set, 'Label')
eval_loader = torch.utils.data.DataLoader( dataset = eval_set , batch_size= batch_size , shuffle = True)

# track metrics over dataset
eval_loss = 0.0
eval_counter = 0.0
eval_accuracy = 0.0
eval_precision = 0.0
eval_recall = 0.0
    
eval_tp = 0.0
eval_tn = 0.0
eval_fp = 0.0
eval_fn = 0.0




# loop through eval data
for i, (images, labels) in enumerate(eval_loader):
    outputs = model(images)
    loss = loss_func(outputs, labels)
    eval_loss += loss.item()
    for i in range(0, len(labels)):
        temp_label = labels[i]
        temp_pred = outputs[i, 1]
      
        if temp_pred > loss_adj_conf_thresh:
            temp_pred = 1.0
        else:
            temp_pred = 0.0
    
        if float(temp_pred) - float(temp_label) == 0:
            eval_accuracy += 1.0

        eval_counter += 1.0

        if float(temp_label) == 1.0 and float(temp_pred) == 1.0:
            eval_tp += 1.0

        if float(temp_label) == 0.0 and float(temp_pred) == 0.0:
            eval_tn += 1.0

        if float(temp_label) == 0.0 and float(temp_pred) == 1.0:
            eval_fp += 1.0

        if float(temp_label) == 1.0 and float(temp_pred) == 0.0:
            eval_fn += 1.0

eval_accuracy = eval_accuracy / eval_counter
try:
    eval_precision = eval_tp / \
        (eval_tp + eval_fp)
except ZeroDivisionError:
    eval_precision = None

try:
    eval_recall = eval_tp / \
        (eval_tp + eval_fn)
except ZeroDivisionError:
    eval_recall = None

print("Accuracy = " + str(eval_accuracy))
print("Loss = " + str(eval_loss))
print("Precision = " + str(eval_precision))
print("Recall = " + str(eval_recall))

# Discussion

Random forest model had the best overall performance in our project. We ended up with a predicting model, a random forest model, with 0.83 precision, 0.73 recall, and 0.91 accuracy on our testing set, which exceeds almost all current published no-show prediction models. However, considering the real world application, though the accuracy is satisfying, the precision and recall are still not good enough. We tried all kinds of stuff to improve precision and recall, yet no big improvement was achieved. We identified two major limitations our models were suffering from that might account for the not-good-enough precision and recall. First, we only have 13 independent variables in our dataset. That means too few features were put into our models. Second, more than 50% patients only showed up once in our dataset thus we actually lacked more than half of the population's previous no-show information, which according to other studies is super important as a predictor. Due to the two major limitations in the dataset, we had difficulty in reaching better precision and recall. In the future studies, after including more features as well as having a longer time frame in the datasets, we believe a model with better precision and recall could be achieved which could assist designing interventions to address the appointment no-show issue to obtain an optimal allocation of scarce medical resources.